**Author: Eric Zeng**

**Email: ericzengsearchll@gmail.com**

**Note:**

This file is about construct minimium variance portfolio using CHF, GBP, JPY from FRED

In [36]:
import pandas as pd
import qrbook_funcs as qf
import numpy as np
from scipy import optimize
# Get 3 currencies until the end of Dec 29, 2017
# From sample covariance matrix and do simple efficient frontier calculations

In [55]:
def get_minimum_variance_portfolio(startday=None, lastday=None, display=True):
    """
    @startday: optional, 'YYYY-MM-DD' dates
    @lastday: optional, 'YYYY-MM-DD' dates
    Return: 
        The mean of return of each instrument
        The covariance matrix of return of instruments
    """
    # Swiss Francs, pound sterling, Japanese Yen
    seriesnames = ['DEXSZUS', 'DEXUSUK', 'DEXJPUS']
    cdates, ratematrix = qf.GetFREDMatrix(seriesnames, startdate=startday, enddate=lastday)
    
    # Convert levels to log-returns
    # First take logs of the currency levels
    # Currency exchange rates are usually expressed in the direction
    # that will make the rate > 1
    # Swissie and yen are in currency/dollar, but
    # pounds is in dollar/currency. Reverse signs
    # so everything is in dollar/currency

    # Do each currency separately to account for separate missing data patterns
    # dlgs is a list of lists of length 3 corresponding to the 3 currencies
    # The value in dlgs is nan if there is missing data for the present or previous day's observation
    # Otherwise it is the log of today/yesterday

    multipliers=[-1,1,-1]
    dlgs=[]
    for i in range(len(multipliers)):
        lgrates=[]
        previous=-1
        for t in range(len(ratematrix)):
            if pd.isna(ratematrix[t][i]) or ratematrix[t][i]<=0:
                lgrates.append(np.nan)    # Append a nan
            else:
                if previous < 0:    # This is the first data point
                    lgrates.append(np.nan)
                else:
                    lgrates.append(np.log(ratematrix[t][i]/previous)*multipliers[i])
                previous=ratematrix[t][i]
        dlgs.append(lgrates)

    # dlgs is the transpose of what we want - flip it
    dlgs=np.transpose(dlgs)

    # Delete any time periods that don't have data
    lgdates=[]
    difflgs=[]
    for t in range(len(dlgs)):
        if all(pd.notna(dlgs[t])):
            #include this time period
            difflgs.append(dlgs[t])
            lgdates.append(cdates[t])

    # Mean vector and covariance matrix are inputs to efficient frontier calculations
    d=np.array(difflgs)
    m=np.mean(d,axis=0)
    c=np.cov(d.T)

    # display the output
    # vectors and matrices are in fractional units;
    #    fraction*100=percent
    #    fraction*10000=basis point
    #    (fraction^2)*10000=percent^2
    if display:
        np.set_printoptions(precision=4)
        print("Means:",m*10000,"bps/day")
        print("(CHF, GBP, JPY)\n")
        print("  ",c[0]*10000)
        print("C=",c[1]*10000,"    (4.20)")
        print("  ",c[2]*10000)
        print(f'(%/day)\N{SUPERSCRIPT TWO} units')
        print("  ")
        print("From",lgdates[0],"to",lgdates[-1],"(",len(lgdates),"observations)")
        print('-------------------')
        print()
        
    return m, c

def portfolio_annualized_performance(weights, meanReturns, covReturnMtx):
    """
    Calculate the standard deviation and returns of a portfolio
    @weights: 1-d array
    @meanReturns: 1-d array
    @covReturnMtx: 2-d array
    Return:
        The standard deviation of the portfolio
    """
    returns = np.sum(meanReturns*weights) * 252 # annualized return
    std = np.sqrt(np.dot(weights.T, np.dot(covReturnMtx, weights))) * np.sqrt(252) # standard deviation
    return std, returns

def portfolio_volatility(weights, meanReturns, covReturnMtx):
    """
    Calculate the standard deviation of the portfolio as the volatility
    @weights: 1-d array
    @meanReturns: 1-d array
    @covReturnMtx: 2-d array
    Return:
        The standard deviation of the portfolio
    """
    std, _ = portfolio_annualized_performance(weights, meanReturns, covReturnMtx)
    return std

def min_volatility(meanReturns, covReturnMtx):
    """
    Calculate the possible minimum variance to achieve given means of return 
    and covariance matrix of returns
    @meanReturns: 1-d array
    @covReturnMtx: 2-d array
    Return:
        The set up to achieve the minimum variance
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covReturnMtx)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound for i in range(numAssets))
    
    weights0 = [1 / numAssets for i in range(numAssets)] # initalize weights as evenly distributed
    # Optimize
    minVolatilitySetUp = optimize.minimize(portfolio_volatility, weights0, args=args, 
                                   method='SLSQP', bounds=bounds, constraints=constraints)
    return minVolatilitySetUp

def calculate_efficient_frontier(seriesnames, meanReturns, covReturnMtx, numPortfolios=10000, display=True):
    """
    Caculate the efficient frontier for the portfolio
    @meanReturns: 1-d array
    @covReturnMtx: 2-d array
    @numPortfolios: default 10000
    @display: boolean
    Return:
        display a table of return, volatility, and allocation
    """
    minVolatilitySetUp = min_volatility(meanReturns, covReturnMtx)
    stdMinPrtf, returnMin = portfolio_annualized_performance(minVolatilitySetUp['x'], meanReturns, covReturnMtx)
    minVolatilityAlloc = pd.DataFrame(minVolatilitySetUp['x'], index=seriesnames, columns=['allocation percentage'])
    minVolatilityAlloc['allocation percentage'] = [round(i * 100, 2) for i in minVolatilityAlloc['allocation percentage']]
    minVolatilityAlloc = minVolatilityAlloc.T
    
    if display:
        print ("Minimum Volatility Portfolio Allocation Percentage\n")
        print ("Annualised Return:", round(returnMin,2))
        print ("Annualised Volatility:", round(stdMinPrtf,2))
        print ("\n")
        print (minVolatilityAlloc)
        print ("--------------------------------")
        


In [50]:
# ending on Dec 29, 2017
means20191229, covReturnMtx20191229 = get_minimum_variance_portfolio(lastday='2017-12-29')

# just 2018 data (250 days)
means2018, covReturnMtx2018 = get_minimum_variance_portfolio(startday='2018-01-01', 
                                                             lastday='2018-12-31')


Means: [ 1.2476 -0.5009  0.98  ] bps/day
(CHF, GBP, JPY)

   [0.5276 0.2528 0.2239]
C= [0.2528 0.3592 0.1183]     (4.20)
   [0.2239 0.1183 0.4209]
(%/day)² units
  
From 1971-01-05 to 2017-12-29 ( 11787 observations)
-------------------

Means: [-0.4703 -2.5494  0.9014] bps/day
(CHF, GBP, JPY)

   [0.1338 0.0892 0.0688]
C= [0.0892 0.2402 0.0361]     (4.20)
   [0.0688 0.0361 0.1579]
(%/day)² units
  
From 2018-01-03 to 2018-12-31 ( 248 observations)
-------------------



**Note:**

The Swiss franc and the Japanese yen have appreciated versus the dollar since 1971 to Dec 29, 2017, giving positive daily mean returns. The pound has depreciated.

The Swiss franc and the Pound sterling have depreciated versus the dollar in 2018 (248 trading days), giving negative daily mean returns. The Japanese yen has appreciated.

In [56]:
# The minimum variance portfolio ending on Dec 29, 2017
calculate_efficient_frontier(["CHF", "GBP", "JPY"], means20191229, covReturnMtx20191229)

Minimum Volatility Portfolio Allocation Percentage

Annualised Return: 0.01
Annualised Volatility: 0.08


                        CHF    GBP    JPY
allocation percentage  4.22  53.12  42.66
--------------------------------


In [57]:
# The minimum variance portfolio in 2018 
calculate_efficient_frontier(["CHF", "GBP", "JPY"], means2018, covReturnMtx2018)

Minimum Volatility Portfolio Allocation Percentage

Annualised Return: -0.01
Annualised Volatility: 0.05


                         CHF    GBP    JPY
allocation percentage  39.08  20.46  40.45
--------------------------------


**Note:**

Portfolio One -> The minimum variance portfolio from 1971-01-05 to 2017-12-29

Portfolio Two -> The minimum variance portfolio in 2018 

**Weights**

The Portfolio One focuses on GBP and JPY. The weights of GBP is small.

The Portfolio Two's weights for each currency is relatively even. 

**Variance**

The annualized volatility of Portfolio One is a bit larger than the the annualized volatility of Portfolio Two.